In [1]:
import pandas as pd
import numpy as np


In [2]:
df_lgbm = pd.read_csv("sub_42_4.csv", index_col=None)
df_lgbm_hosei = pd.read_csv("sub_42_4_hosei.csv", index_col=None)
df_lgbm_q_50 = pd.read_csv("sub_42_4_q_50-50-50.csv", index_col=None)
df_lgbm_q_50k = pd.read_csv("sub_42_4_q_50000-50000-50000.csv", index_col=None)
df_svr = pd.read_csv("sub_svr_42_4.csv", index_col=None)
df_mlp = pd.read_csv("sub_mlp_42_4.csv", index_col=None)


In [3]:
cols = ["pollen_utsunomiya", "pollen_chiba", "pollen_tokyo"]


In [4]:
_df = pd.read_csv("../input/train_v2.csv", index_col=None)


In [5]:
def getNearestValue(list, num):
    """
    https://qiita.com/icchi_h/items/fc0df3abb02b51f81657
    概要: リストからある値に最も近い値を返却する関数
    @param list: データ配列
    @param num: 対象値
    @return 対象値に最も近い値
    """

    # リスト要素と対象値の差分を計算し最小値のインデックスを取得
    idx = np.abs(np.asarray(list) - num).argmin()
    return list[idx]


In [6]:
lgbm_score = 12.22388
lgbm_hosei_score = 12.46269
catboost_score = 12.36318
lgbm_q_50_score = 12.26368
lgbm_q_50k_score = 15.25373
lgbm_q_20_score = 13.41791
svr_score = 12.44279
mlp_score = 12.71144	


In [7]:
df = pd.DataFrame()
df["datetime"] = df_lgbm["datetime"]
for col in cols:
    pollen_list = list(set(_df[col].tolist()))    

    if col == "pollen_utsunomiya":
        print(col)
        score_list = [lgbm_score, lgbm_hosei_score, lgbm_q_50_score, svr_score, mlp_score]        

        s = sum([100 - x for x in score_list])
        w_lgbm = (100 - lgbm_score) / s
        w_lgbm_hosei = (100 - lgbm_hosei_score) / s
        w_lgbm_q_50 = (100 - lgbm_q_50_score) / s
        w_svr = (100 - svr_score) / s
        w_mlp = (100 - mlp_score) / s

        df[col] = (
            (df_lgbm[col] * w_lgbm)
            + (df_lgbm_hosei[col] * w_lgbm_hosei)
            + (df_lgbm_q_50[col] * w_lgbm_q_50)
            + (df_svr[col] * w_svr)
            + (df_mlp[col] * w_mlp)
        )

    elif col == "pollen_tokyo":
        print(col)        
        score_list = [lgbm_hosei_score, lgbm_q_50k_score]
        s = sum([100 - x for x in score_list])        
        w_lgbm_hosei = (100 - lgbm_hosei_score) / s        
        w_lgbm_q_50k = (100 - lgbm_q_50k_score) / s

        df[col] = (            
            (df_lgbm_hosei[col] * w_lgbm_hosei)            
            + (df_lgbm_q_50k[col] * w_lgbm_q_50k)
        )

    elif col == "pollen_chiba":
        print(col)        
        score_list = [lgbm_hosei_score, lgbm_q_50k_score]
        s = sum([100 - x for x in score_list])        
        w_lgbm_hosei = (100 - lgbm_hosei_score) / s        
        w_lgbm_q_50k = (100 - lgbm_q_50k_score) / s
        df[col] = (            
            (df_lgbm_hosei[col] * w_lgbm_hosei)            
            + (df_lgbm_q_50k[col] * w_lgbm_q_50k)
        )
    else:
        raise ValueError("unk")

    df[col] = np.array([getNearestValue(pollen_list, v) for v in df[col]])

df = df[["datetime"] + cols]
df.to_csv("sub_ensemble_6.csv", index=None)


pollen_utsunomiya
pollen_chiba
pollen_tokyo


- utにmlpを追加 11.94527 -> sub6
- utにcatboost追加 11.96517
- cb/tk に svr_q_50k 追加　12.12438
- utにsvr追加 11.94527 -> sub_5
- utにq_20を追加 11.96517
- ut は hoseiと普通のやつのみ 12.00498
- cb/tk は q_50k のみ 12.08458
- cb/tk は hosei と q_50k 11.96517
- utはhoseのみ 12.28358
- utのみq20を入れる 12.10448
- ut以外q50k入れる 12.04478
- svr 入れる 12.16418
- q50にしたやつ  12.14428
- catboost入れる 12.18408
- LBのスコアで重みづけてアンサンブル 12.14428

In [8]:
df["pollen_utsunomiya"].sum(), df["pollen_tokyo"].sum(), df["pollen_chiba"].sum()


(3244.0, 1440.0, 1700.0)

In [9]:
total_pollen_2020_ut = 47848.946232729206
total_pollen_2020_tk = 28232.168622513003
total_pollen_2020_cb = 32978.780406463025

test_term_pollen_rate_ut = 13.401466765933876
test_term_pollen_rate_tk = 15.31909333922222
test_term_pollen_rate_cb = 12.843262552256846

test_term_pollen_ut = total_pollen_2020_ut * (test_term_pollen_rate_ut / 100)
test_term_pollen_tk = total_pollen_2020_tk * (test_term_pollen_rate_tk / 100)
test_term_pollen_cb = total_pollen_2020_cb * (test_term_pollen_rate_cb / 100)


In [10]:
test_term_pollen_ut, test_term_pollen_tk, test_term_pollen_cb


(6412.460627228774, 4324.912262969375, 4235.551354134283)

- lgbm_hosei/ lgbm: LB: 12.14428